# Lifecycle

## Definitions

It refers to the lifecycle of an object(from birth to death). The life of an object refers to the RAM management.

## Lifecycle Monitoring

* `__new__`
  * When we create a new object, this method will allocate resources for the object.
  * By Interception of the method, we can control the lifecycle of the object.  
* `__init__`
  * ...
* `__del__`
  * This function will be called automatically when the object is released

In [7]:
class Person:
    # def __new__(cls, *args,**kwargs):
    #     print('Your operation of creating a new object is intercepted')
    #     # TODO: You should create the object below
    
    def __init__(self, *args, **kwargs):
        print("Initializing the object")
        self.name  = 'lol'
    
    # This function will be called automatically when the object is released
    def __del__(self):
        print("I haven't deleting the object")
    pass

p = Person()
print(p)
print(p)
del p # __del__ will also be called when we delete the object


Initializing the object
I haven't Deleting the object
I haven't deleting the object


***Example:***  How to monitor the lifecycles of a instance.  
When instance created, add one to the counter and minus when instance deleted

In [47]:

class Person(object):
    __Counter_Person = 0
    def __init__(self, *args, **kwargs):
        print("Counter + 1")
        # Counter_Person += 1
        Person.__Counter_Person += 1

    def __del__(self):
        print('Counter - 1 ')
        self.__class__.__Counter_Person -= 1

    @classmethod
    def log(cls):
        print('Person Counter = ',cls.__Counter_Person)
    pass
    # @staticmethod
    # def log():
    #     print('Person Counter = ',Person.__Counter_Person)
    # pass

p3 = Person()
p4 = Person()
# print(Counter_Person)
Person.log()
print(p3)
Person.log()
del p3
# del p4
Person.log()


Counter + 1
Counter + 1
Counter - 1 
Person Counter =  2
Person Counter =  2
Counter - 1 
Person Counter =  1


##### Note: Sometimes you will identify:


```python
    p3 = Person()
    p4 = Person()
    # print(Counter_Person)
    Person.log()
    print(p3)
    Person.log()
    del p3
    # del p4
    Person.log()
```
The output

```text
    Counter + 1
    Counter + 1
    Counter - 1 
    # This is because it will delete the last p4 in the next run. 
    # This will not happen in the first run
    Person Counter =  2
    <__main__.Person object at 0x000001DB05517A90>
    Person Counter =  2
    Counter - 1 
    Person Counter =  1
```


## Storage Management

### Storage
1. Everything in Python is an object. No elementry data type.
2. Every object will be allocated a space in the ram.
   1. Dynamic
   2. Will return the address to outside('quote')
      * `id()`: by decimal
      * `hex()`: by hexadecimal
3. Small number and string like 1,2,3,4. This will be the same place in ram
4. For Container objects, this will be a quote, but not the object itself.

In [50]:
# 3
number1 = 2
number2 = 2
print(id(number1) == id(number2))

True


##### Note: Container objects
An object that can refer other objects

Container objects" typically refer to data structures used in programming and computer science for storing and organizing data. Different programming languages provide various types of container objects. Here are some common container objects:

* Array: An ordered collection of elements accessed by index.  
* List: Similar to an array but dynamically resizable, often providing more operations and features.  
* Set: An unordered collection of unique elements.  
* Dictionary (or Map): Stores key-value pairs, allowing access to values through keys.  
* Queue: A data structure that follows the First-In-First-Out (FIFO) principle.  
* Stack: A data structure that follows the Last-In-First-Out (LIFO) principle.  
* Tuple: An ordered, immutable data structure.  
* Linked List: A data structure consisting of nodes, each containing data and a reference to the next node.  
* Heap: Used for priority queue implementations, often employed in heap sort.  
* Tree: A hierarchical data structure with a root node, child nodes, and leaf nodes, representing hierarchical relationships.  

### Cache management


1. Reference counter
   * An object will remember its reference number
   * Whwn to +1 and -1
   * What is the problem: circular references
2. Garbage collection
   * Find the circular reference and del all the object
   * How to find
   * How to improeve its functionality
3. Access reference counter
   * import sys
   * .getrefcount() 

#### Reference counter

Reference counter
   * An object will remember its reference number
   * +1
     * Be created
     * Be refered
     * Be sent into a function
     * Be an element and stored in a container object 
   * -1
     * del p1
     * p1 = 123 (p1 is assigned to another object)
     * An object leaves its domain(A function finishes)
     * An object leaves its container

In [60]:
import sys
class Person:
    pass


p1 = Person()
print(sys.getrefcount(p1)) 
# When you put it in the function it will +1. 
# When the function finishes opeartion it will -1
# So the value printed out will be 1 larger
p2 = p1 # Assignment operation(赋值操作)
print(sys.getrefcount(p1))
print(p1==p2) # They are the same
del p2 
print(sys.getrefcount(p1))
del p1
# print(sys.getrefcount(p1))


2
3
True
2


##### A problem: Circular reference

When 2 object refer each other but you cut off all the outer reference. It will not be released which will cause the ram overflow.

In [103]:
import objgraph

class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()
print(objgraph.count('Person'))
print(objgraph.count('Dog'))

p.pet = d
d.master = p

del p
del d

print(objgraph.count('Person'))
print(objgraph.count('Dog'))

3
2
3
2


Garbage collection will be done like 30 times for 0 generation

#### Grabage collection

##### How to find the circular reference? Specially for container objects

1. Collect all the container objects and use a doubly linked list(双向链表) to refer
2. For every container object, by `gc_refs` to count reference number
3. For every container object A, find the container object B that A point to and then count(B) -1
4. When A = 0, recycle. 

##### Note Doubly linked list

The term for "双向链表" in English is "doubly linked list." In a doubly linked list, each node contains two pointers: one pointing to the previous node (predecessor) and another pointing to the next node (successor). This structure allows for bidirectional traversal at any node in the linked list. Compared to a singly linked list, it provides more flexibility but requires additional space to store the extra pointers.

##### Problem: energy consumption. 
You need to check a lot of things.

***Assumption:***  
* The bigger object, the longer it will exist.
* The more time it was checked, the longer it will exist.  

***Generational garbage collection(分代回收)***  
天上一天地下一年  
1. Creation: 0 generation
2. Exist after one scan -> +1 generation
3. Their frequency is different 

import gc
gc.get_thershold()
gc.set_thershold()

In [105]:
import gc
gc.get_threshold()

(700, 10, 10)

* 700: Garbage collection will start, when Number of Created object - Number of released objects = 700  
* 10: After 10 times of 0 th, 0&1 generations will be scanned together  
* 10: After 10 times of 1 th, 1&2 generations will be scanned together  

We can set those numbers larger to improve performance.

##### When to collect: frequency and start conditions

1. Automatic
   * Start
     * `gc.enable()`
     * `gc.disable()`
     * `gc.get_thershold()`
2. Manual
   * 

In [107]:
# Automatic
import gc

print(gc.isenabled())
gc.disable()
print(gc.isenabled)
print(gc.enable())


True
<built-in function isenabled>
None


In [140]:
# Manual

import gc
import objgraph

class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()

p.pet = d
d.master = p

del p
del d
# We need to clear the stuff manualy
gc.collect(0)
# 0: only 0 th generation 
# 1: 0 & 1
# 2: 0 & 1 & 2
print(objgraph.count('Person'))
    

1


##### How to do it in reality

***Weak reference:***  
In Python 2, if two containers involved in a circular reference are not released because one of them modifies the release conditions via __del__, neither of them will be released. For a cross-version solution, you can use weak references.  
In the context of circular references and the challenges posed by the __del__ method in Python 2, it's advisable to utilize weak references. Weak references in Python provide a way to maintain references to objects without preventing them from being garbage collected. Unlike strong references, weak references do not influence the reference count of the object they point to, allowing circular references to be broken, and objects to be released when they are no longer in use.



In [155]:
import weakref
import gc
import objgraph

class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()

p.pet = d
# Accessing the object from the weak reference
# And we can break the cycle
d.master = weakref.ref(p)

d1 = Dog()
d1.master = p
d2 = Dog()
d2.master = p
# What would happen if we have a lot of complicated objects connected to each other
# p.pets = {'Dog1': weakref.ref(d1), 'Dog2': weakref.ref(d2)} # Method 1
weakref.WeakValueDictionary({'Dog1': d1, 'Dog2': d2}) # Method 2 

del p
del d
# We need to clear the stuff manualy
# gc.collect(0)
# 0: only 0 th generation 
# 1: 0 & 1
# 2: 0 & 1 & 2
print(objgraph.count('Person'))

2


***Point it to none:***

In [148]:
class Person:
    pass

class Dog:
    pass

p = Person()
d = Dog()

p.pet = d
# Accessing the object from the weak reference
# And we can break the cycle
d.master = weakref.ref(p)

p.pet = None
del p
del d
print(objgraph.count('Person'))

1
